# Graph Data dase representation: First scenario:

Now we have a csv file that contains all realtionships that in each usecase, and the next step is to make a graph database representation usin the neo4j AURA DB, by using its query language `Cypher` with its interface with python to automate the operation of Creating and editing the reationships.

In [1]:
# Start neo4j 
!chmod a+x neo4j-desktop-1.4.15-x86_64.AppImage; ./neo4j-desktop-1.4.15-x86_64.AppImage

chmod: cannot access 'neo4j-desktop-1.4.15-x86_64.AppImage': No such file or directory
/bin/bash: ./neo4j-desktop-1.4.15-x86_64.AppImage: No such file or directory


In [131]:
import pandas as pd
import numpy as np
from neo4j import GraphDatabase
import logging
from neo4j.exceptions import ServiceUnavailable

## Importing CSV realtionship file and Editing the components names:

In [2]:
relations = pd.read_csv("./figure_parse.csv")

In [3]:
relations.head()

,source,target,message,class,id
0,Knowledge base,AN component-N,exchange of knowledge,critical relationship,UC001
1,AN Component-1,AN component-N,exchange of knowledge,critical relationship,UC001
2,Knowledge base,AN Component-1,Report,critical relationship,UC001
3,Knowledge base,AN Component-1,exchange of knowledge,critical relationship,UC001
4,Management domain,Knowledge base,import of knowledge,critical relationship,UC001


After we Saw the tablem we have found some inconsitancy in names of components, and after that we have the following inconvience:
- "Sandbox" -> "AN Sandbox",
- "Sandbox environment" -> "AN Sandbox",
- "Industry vertical solution providers" -> "Industry vertical solution provider",
- "Closed loops" -> "Controller",
- "Closed loop automation frameworks" -> "CL automation framework",
- "Control loop" -> "Controller",
- "Control loop app store(public)" -> "Controller repository",
- "Control loop designer" -> "Controller designer",
- "REx:: AN Orchestrator" -> "AN orchestrator",
- "AN Orchestrator" -> "AN orchestrator",
- "AN component-1" -> "AN Component-1"

In [4]:
edits = {
    "Sandbox":"AN Sandbox",
    "Sandbox environment":"AN Sandbox",
    "Industry vertical solution providers":"Industry vertical solution provider",
    "Closed loops":"Controller",
    "Closed loop automation frameworks":"CL automation framework",
    "Control loop":"Controller",
    "Control loop app store(public)":"Controller repository",
    "Control loop designer":"Controller designer",
    "REx:: AN Orchestrator":"AN orchestrator",
    "AN Orchestrator":"AN orchestrator",
    "AN component-1": "AN Component-1"
}

In [5]:
for key in edits:
    relations.loc[relations.source == key, "source"] = edits[key]
    relations.loc[relations.target == key, "target"] = edits[key]

In [6]:
sources = list(relations.source.unique())
targets = list(relations.target.unique())
all_ = sources + targets
components = set(np.array(all_).flatten())

In [7]:
len(components)

45

In [8]:
components

{'3rd party developer',
 'AN',
 'AN Component-1',
 'AN Sandbox',
 'AN component-2|human',
 'AN component-N',
 'AN orchestrator',
 'AN_1',
 'AN_2',
 'AN_3',
 'Anatomy engine',
 'BS1',
 'BS2',
 'BS3',
 'BS4',
 'Backend',
 'CL automation framework',
 'Controller',
 'Controller designer',
 'Controller repository',
 'Deployment manager',
 'End-user communities',
 'Ex manager',
 'FLAML controller',
 'Industry vertical applications',
 'Industry vertical applications @ edge',
 'Industry vertical solution provider',
 'Intelligent AR app\n(Glasses)',
 'Intelligent maintenance assistance system',
 'Inventory systems',
 'Knowledge base',
 'ML layer',
 'Management domain',
 'NAO',
 'NAO @ edge',
 'Network service outside the domain',
 'Network service within domain',
 'Network services',
 'OpenCN (repo)',
 'Operations team',
 'Private 5G deployment @ edge',
 'Service LCM',
 'Workflow manager',
 'Workflow mediator',
 'human'}

# REF Code with added link prediction functions:

In [2]:
class App:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        # Don't forget to close the driver connection when you are finished with it
        self.driver.close()

    #Vishnu: 1 Aug 2022: uses labels instead of properties
    #Vishnu: 1 Aug 2022: Invariant: only 1 actor with 1 name.
    #        (Note that rel still uses properties and not labels)
    def create_node_with_usecase_label(self, actor_name, usecase_id):
        with self.driver.session() as session:
            node_already_exists = session.read_transaction(
                self._find_and_return_existing_node_label, actor_name)
            if not node_already_exists:  
                #node itself doesnt exist (not to mention the use case label), Lets add.
                result = session.write_transaction(
                    self._create_and_return_node_label, actor_name, usecase_id)
                for row in result:
                    print("Created node: {n1}".format(n1=row['n1name']))
            else:
                for row in node_already_exists:
                    print("Node already exists: {name} in {usecase}".format(name=row['n1name'], 
                                                                 usecase=row['n1usecase']))
                    if not (usecase_id in row['n1usecase']):
                      print("adding label "
                                +usecase_id+ 
                                " in addition to {usecase}".format(usecase=row['n1usecase']))
                      result = session.write_transaction(
                                self._add_usecase_and_return_existing_node_label, actor_name, usecase_id)
                    else:
                      print("ignoring ...")
                    #row has only 1 entry due to invariant. So break here. and exit f()
                    break
    
    #Vishnu: 1 Aug 2022: created this f() to query nodes based on a name
    #                    and return with usecase labels
    @staticmethod
    def _find_and_return_existing_node_label(tx, actor_name):
        query = (
            "MATCH (n1) "
            "WHERE (n1.name ='" + actor_name + "') "
            "RETURN n1.name as n1name, labels(n1) as n1usecase"
        )
        result = tx.run(query)
        try:
            return [{"n1name": row["n1name"],"n1usecase": row["n1usecase"]} 
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    #Vishnu: 1 Aug 2022: created this f() to create nodes with usecase label
    @staticmethod
    def _create_and_return_node_label(tx, actor_name, usecase_id):
            query = (
            "CREATE (n1 :" + usecase_id + " { name: '" + actor_name + "'}) "
            "RETURN n1"
            )
            result = tx.run(query, actor_name=actor_name)
            try:
                return [{"n1name": row["n1"]["name"]}
                        for row in result]
            # Capture any errors along with the query and data for traceability
            except ServiceUnavailable as exception:
                logging.error("{query} raised an error: \n {exception}".format(
                    query=query, exception=exception))
                raise

    #Vishnu: 1 Aug 2022: created this f() to add use case labels to existing nodes
    #                    Note that the existing labels are left there.
    @staticmethod
    def _add_usecase_and_return_existing_node_label(tx, actor_name, usecase_id):
        query = (
            "MATCH "
            "(n1) "
            "WHERE n1.name = '" + actor_name +"' "
            "set n1 :" + usecase_id +" "
            "return n1"
        )
        result = tx.run(query, actor_name=actor_name)
        try:
            return [{"n1": row["n1"]["name"]}
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    #Vishnu: 1 Aug 2022: created this f() to return all use case actors based on labels
    #                    as against properties
    @staticmethod
    def _find_and_return_all_usecase_actors_label(tx, usecase_id):
        query = (
            "MATCH (n) "
            "WHERE '" +usecase_id+ "' in labels(n) "
            "RETURN n.name AS name"
        )
        result = tx.run(query)
        return [row["name"] for row in result]

    #Vishnu: 1 Aug 2022: created this f() to list all nodes with usecase label
    #                    instead of properties
    def find_all_usecase_actors_label(self, usecase_id):
        with self.driver.session() as session:
            result = session.read_transaction(self._find_and_return_all_usecase_actors_label, usecase_id)
            i=0
            for row in result:
                i=i+1
                print("Found actor-"+ str(i) +": {row}".format(row=row))

    #Vishnu: 1 Aug 2022: Created to use label for use case.
    #CAUTION: Assumes that actors already exists with label as use case.
    #So, this has to be called only in conjunction with create_node_with_usecase
    #otherwise you may end up with actors not having use case prop
    #but rels will have use case prop. not a good idea.
    def create_rel_with_usecase_label(self, actor1_name, rel_name, actor2_name, usecase_id):
        with self.driver.session() as session:
            this_rel_already_exists = session.read_transaction(
                self._find_and_return_this_existing_rels, actor1_name, rel_name, actor2_name, usecase_id)
            if (this_rel_already_exists):
                print(rel_name + " already exists with "+ usecase_id + " between "+ actor1_name + " and "+ actor2_name)
                return
            else:
                #we are pretty sure something didnt match
                #this relation does not exist currently between the nodes under the use case, 
                #add it.
                result3 = session.write_transaction(
                    self._create_and_return_rel_label, actor1_name, rel_name, actor2_name, usecase_id)
                #if (not retult3)
                for row in result3:
                    print("Created relation: {actor1} - {rel} - {actor2} in {usecase}".
                            format(actor1=row['n1name'], 
                            rel=row['r1name'],
                            actor2=row['n2name'],
                            usecase=row['r1usecase']))

    #Vishnu: 1 Aug 2022 
    #assumption: nodes already exist
    #modified from _create_and_return_rel to use labels for n.usecase
    @staticmethod
    def _create_and_return_rel_label(tx, actor1_name, rel_name, actor2_name, usecaseid):
            query = (
            "MATCH "
            "(n1), "
            "(n2) "
            "WHERE n1.name = '" + actor1_name + "' AND n2.name = '"+actor2_name+"' "
            " AND '"+ usecaseid+"' in labels(n1) "
            " AND '"+ usecaseid+"' in labels(n2) "
            "CREATE (n1)-[r1:Relation " + "{name: '"+rel_name+"', usecase: '"+usecaseid+"'} ]->(n2) "
            "RETURN n1, r1, n2"
            )
            result = tx.run(query)
            try:
                return [{"n1name": row["n1"]["name"], 
                        "r1name": row["r1"]["name"],
                        "n2name": row["n2"]["name"],
                        "r1usecase": row["r1"]["usecase"]}
                        for row in result]
            # Capture any errors along with the query and data for traceability
            except ServiceUnavailable as exception:
                logging.error("{query} raised an error: \n {exception}".format(
                    query=query, exception=exception))
                raise

    #Vishnu: 1 Aug 2022: This approach uses properties instead of labels for n.usecases.
    #Vishnu: 1 Aug 2022: Invariant: only 1 actor with 1 name.
    #Vishnu: 22 Jun 2022: refactored, uses properties instead of labels
    #Vishnu: 1 Jun 2022: created this f() to create nodes with usecase label
    #Vishnu: 2 Jun 2022: added- check if it already exists.
    #Vishnu: 2 Jun 2022: added- check the usecase duplication, adds a usecase if it doesnt exist.
    #otherwise ignores the command.
    def create_node_with_usecase(self, actor_name, usecase_id):
        with self.driver.session() as session:
            node_already_exists = session.read_transaction(
                self._find_and_return_existing_node, actor_name)
            if not node_already_exists:  
                result = session.write_transaction(
                    self._create_and_return_node, actor_name, usecase_id)
                for row in result:
                    print("Created node: {n1}".format(n1=row['n1name']))
            else:
                for row in node_already_exists:
                    print("Node already exists: {name} in {usecase}".format(name=row['n1name'], 
                                                                 usecase=row['n1usecase']))
                    s=row['n1usecase'].split(',')
                    if not (usecase_id in s):
                      print("adding prop "+usecase_id+ " in addition to "+row['n1usecase'])
                      result = session.write_transaction(
                                self._add_usecase_and_return_existing_node, actor_name, usecase_id)
                    else:
                      print("ignoring ...")
                    #row has only 1 entry due to invariant. So break here. and exit f()
                    break

    #Vishnu: 1 Aug 2022: This approach uses properties instead of labels for usecases.
    #Vishnu: 2 Jun 2022: created this f() to query nodes and return with usecase prop
    @staticmethod
    def _find_and_return_existing_node(tx, actor_name):
        query = (
            "MATCH (n1) "
            "WHERE (n1.name ='" + actor_name + "') "
            "RETURN n1.name as n1name, n1.usecase as n1usecase"
        )
        result = tx.run(query)
        try:
            return [{"n1name": row["n1name"],"n1usecase": row["n1usecase"]} 
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    #Vishnu: 1 Aug 2022: This approach uses properties instead of labels for usecases.
    @staticmethod
    def _create_and_return_node(tx, actor_name, usecase_id):
            query = (
            "CREATE (n1 "+"{ name: '"+actor_name+"', usecase: '"+usecase_id+"'}) "
            "RETURN n1"
            )
            result = tx.run(query, actor_name=actor_name)
            try:
                return [{"n1name": row["n1"]["name"]}
                        for row in result]
            # Capture any errors along with the query and data for traceability
            except ServiceUnavailable as exception:
                logging.error("{query} raised an error: \n {exception}".format(
                    query=query, exception=exception))
                raise

    #Vishnu: 1 Aug 2022: This approach uses properties instead of labels for usecases.
    #Vishnu: 1 Jun 2022: created this f() to add use case prop to existing nodes 
    #CAUTION: does not check for duplication, use create_node_with_prop instead.
    def add_usecase_existing_node(self, actor_name, usecase_id):
        with self.driver.session() as session:
            # Write transactions allow the driver to handle retries and transient errors
            result = session.write_transaction(
                self._add_usecase_and_return_existing_node, actor_name, usecase_id)
            
            # we expect only 1 node. Do we need the for loop below?
            for row in result:
                print("added usecase to node: {n1}".format(n1=row['n1']))

    #Vishnu: 1 Aug 2022: This approach uses properties instead of labels for usecases.
    @staticmethod
    def _add_usecase_and_return_existing_node(tx, actor_name, usecase_id):
        # To learn more about the Cypher syntax, see https://neo4j.com/docs/cypher-manual/current/
        # The Reference Card is also a good resource for keywords https://neo4j.com/docs/cypher-refcard/current/
        query = (
            "MATCH "
            "(n1) "
            "WHERE n1.name = '" + actor_name +"' "
            "set n1.usecase = n1.usecase + '" + "," + "'"+ "+ '"+usecase_id +"' "
            "return n1"
        )
        result = tx.run(query, actor_name=actor_name)
        try:
            return [{"n1": row["n1"]["name"]}
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    #Vishnu: 1 Aug 2022: This approach uses properties instead of labels for usecases.
    #CAUTION: Assumes that actors already exists with property as n.use case.
    #So, this has to be called only in conjunction with create_node_with_usecase
    #otherwise you may end up with actors not having use case prop
    #but rels will have use case prop. not a good idea.
    def create_rel_with_usecase(self, actor1_name, rel_name, actor2_name, usecase_id):
        with self.driver.session() as session:
            this_rel_already_exists = session.read_transaction(
                self._find_and_return_this_existing_rels, actor1_name, rel_name, actor2_name, usecase_id)
            if (this_rel_already_exists):
                print(rel_name + " already exists with "+ usecase_id + " between "+ actor1_name + " and "+ actor2_name)
                return
            else:
                #we are pretty sure something didnt match
                #this relation does not exist currently between the nodes under the use case, 
                #add it.
                result3 = session.write_transaction(
                    self._create_and_return_rel, actor1_name, rel_name, actor2_name, usecase_id)
                #if (not retult3)
                for row in result3:
                    print("Created relation: {actor1} - {rel} - {actor2} in {usecase}".
                            format(actor1=row['n1name'], 
                            rel=row['r1name'],
                            actor2=row['n2name'],
                            usecase=row['r1usecase']))
    
    #Vishnu: 1 Aug 2022: at this moment this f() uses labels instead of properties for n.usecases.
    #                    Note that rels still uses properties.
    #        1 Aug 2022: TBD: create a property-version (as against label) of the same thing.
    #Vishnu: 21 Jun 2022: created this f() to create relationships with usecase label
    #if this works, this is the only function that needed to be called.
    def create_actors_relationship_with_usecase(self, actor1_name, rel_name, actor2_name, usecase_id):
            #process actor1
            self.create_node_with_usecase_label(actor1_name, usecase_id)
            #process actor2
            self.create_node_with_usecase_label(actor2_name, usecase_id)
            #process relation
            self.create_rel_with_usecase_label(actor1_name, rel_name, actor2_name, usecase_id)

    #Vishnu: 23 June 2022: created
    #looks for a rel under a use case between 2 given nodes.
    #normally, we expect only 0 or 1 unique entry.
    @staticmethod                
    def _find_and_return_this_existing_rels(tx, actor1_name, rel_name, actor2_name, usecase_id):
        query = (
            "MATCH (n1)-[r1]-(n2) "
            "WHERE (n1.name ='" + actor1_name + "') "
            "and   (n2.name ='" + actor2_name + "') "
            "and   (r1.usecase CONTAINS '"+ usecase_id+ "') "
            "and   (r1.name ='" + rel_name + "') "
            "RETURN n1.name as n1name, r1.name as r1name, n2.name as n2name, r1.usecase as r1usecase"
        )
        result = tx.run(query)
        return [row["r1name"] for row in result]
        
    
    #Vishnu: 21 June 2022: created this supporting f()
    #return for all relationship between given two nodes.
    @staticmethod                
    def _find_and_return_all_existing_rels_for_uc(tx, actor1_name, actor2_name, usecase_id):
        query = (
            "MATCH (n1)-[r1]-(n2) "
            "WHERE (n1.name ='" + actor1_name + "') "
            "and   (n2.name ='" + actor2_name + "') "
            "and   (r1.usecase CONTAINS '"+ usecase_id+ "') "
            "RETURN n1.name as n1name, r1.name as r1name, n2.name as n2name, r1.usecase as r1usecase"
        )
        result = tx.run(query)
        try:
            return [{
                        "n1name": row["n1name"],
                        "n2name": row["n2name"],
                        "r1name": row["r1name"],
                        "r1usecase": row["r1usecase"],
                    } 
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    #Vishnu: 21 June 2022: created this supporting f()
    #adds a name for an existing relationship between given two nodes.
    @staticmethod                
    def _add_usecase_and_return_existing_rel(tx, actor1_name, rel_name, actor2_name, usecase_id):
        query = (
            "MATCH (n1)-[r1]-(n2) "
            "WHERE (n1.name ='" + actor1_name + "') "
            "and   (n2.name ='" + actor2_name + "')"
            "set    r1.usecase = r1.usecase + '" + "," + "'"+ "+ '"+usecase_id +"' "
        )
        result = tx.run(query)

    
    #Vishnu: 30 june 2022: created
    #CAUTION: if the property already exists, 
    #this overwrites the current value.
    @staticmethod
    def _write_property_and_return_existing_node(tx, actor_name, propertyName, propertyVal):
        query = (
            "MATCH "
            "(n1) "
            "WHERE n1.name = '" + actor_name +"' "
            "set n1."+propertyName+" = '"+propertyVal +"' "
            "return n1"
        )
        result = tx.run(query, actor_name=actor_name)
        try:
            return [{"n1": row["n1"]["name"]}
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    #Vishnu: 30 june 2022: created
    #CAUTION: if the property already exists, 
    #this overwrites the current value.
    @staticmethod
    def _write_property_and_return_existing_rel(tx, actor1_name, rel_name, actor2_name, 
                                                propertyName, propertyVal):
        query = (
            "MATCH (n1)-[r1]-(n2) "
            "WHERE (n1.name ='" + actor1_name + "') "
            "and   (n2.name ='" + actor2_name + "') "
            "and   (r1.name ='" + rel_name + "') "
            "set   r1."+propertyName+" = '"+propertyVal +"' "
            "return r1"
        )
        result = tx.run(query)
        try:
            return [{"r1": row["r1"]["name"]}
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    #Vishnu: 22 June 2022: created
    #assumption: nodes already exist
    @staticmethod
    def _create_and_return_rel(tx, actor1_name, rel_name, actor2_name, usecaseid):
            query = (
            "MATCH "
            "(n1), "
            "(n2) "
            "WHERE n1.name = '" + actor1_name + "' AND n2.name = '"+actor2_name+"' "
            " AND n1.usecase CONTAINS '"+ usecaseid+"' "
            " AND n2.usecase CONTAINS '"+ usecaseid+"' "
            "CREATE (n1)-[r1:Relation " + "{name: '"+rel_name+"', usecase: '"+usecaseid+"'} ]->(n2) "
            "RETURN n1, r1, n2"
            )
            result = tx.run(query)
            try:
                return [{"n1name": row["n1"]["name"], 
                        "r1name": row["r1"]["name"],
                        "n2name": row["n2"]["name"],
                        "r1usecase": row["r1"]["usecase"]}
                        for row in result]
            # Capture any errors along with the query and data for traceability
            except ServiceUnavailable as exception:
                logging.error("{query} raised an error: \n {exception}".format(
                    query=query, exception=exception))
                raise

    #Vishnu: 1 Jun 2022: created this f() to list all nodes with usecase label
    def find_all_usecase_actors(self, usecase_id):
        with self.driver.session() as session:
            result = session.read_transaction(self._find_and_return_all_usecase_actors, usecase_id)
            i=0
            for row in result:
                i=i+1
                print("Found actor-"+ str(i) +": {row}".format(row=row))

    @staticmethod
    def _find_and_return_all_usecase_actors(tx, usecase_id):
        query = (
            "MATCH (n) "
            "WHERE (n.usecase = '"+usecase_id+"') "
            "RETURN n.name AS name"
        )
        result = tx.run(query)
        return [row["name"] for row in result]

    #Vishnu: 21 Jun 2022: created this f() to cleanup the DB
    #CAUTION: will delete everything
    def cleanup_db(self):
        with self.driver.session() as session:
            result = session.read_transaction(
                self._count_nodes_in_db)
            for row in result:
                print("Found {row} actors, deleting! ".format(row=row['count']))
            result = session.write_transaction(
                self._cleanup_db)

    #Vishnu: 22 June 2022: created this f() 
    #(over)writes a prop to a node
    def writePropToNode(self, actor_name, propertyName, propertyVal):
        with self.driver.session() as session:
            result = session.write_transaction(
                self._write_property_and_return_existing_node, 
                    actor_name, 
                    propertyName, 
                    propertyVal)
        
            # we expect only 1 node. Do we need the for loop below?
            for row in result:
                print("added prop "+
                        propertyName+
                        " = "+
                        propertyVal+
                        " to node: {n1}".format(n1=row['n1']))

    #Vishnu: 22 June 2022: created this f() 
    #(over)writes a prop to a rel
    def writePropToRel(self, a1, rel_name, a2, propertyName, propertyVal):
        with self.driver.session() as session:
            result = session.write_transaction(
                self._write_property_and_return_existing_rel, 
                    a1, rel_name, a2,
                    propertyName, 
                    propertyVal)
        
            # we expect only 1 node. Do we need the for loop below?
            for row in result:
                print("added prop "+
                        propertyName+
                        " = "+
                        propertyVal+
                        " to rel: {r1}".format(r1=row['r1']))

    @staticmethod
    def _cleanup_db(tx):
        query = (
            "MATCH (n) "
            "detach delete n"
        )
        result = tx.run(query)
        try:
            return 
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    @staticmethod
    def _count_nodes_in_db(tx):
        query = (
            "MATCH (n) "
            "return count(n) as count"
        )
        result = tx.run(query)
        try:
            return [{"count": row["count"]} 
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise
            
            
    ################################################################################################   
    """
    Link prediction functions by Tarek:
    """
    ################################################################################################

    @staticmethod
    def _creat_projection(tx, proj):
        query = (
            "CALL gds.graph.project('" + proj +"', '*', {Relation: {orientation: 'UNDIRECTED'}}) "
        )
        result = tx.run(query)
    
    def project(self):
         with self.driver.session() as session:
            result = session.write_transaction(
                self._creat_projection)

    @staticmethod
    def _make_pipe(tx, pipe):
        query = (
            "CALL gds.beta.pipeline.linkPrediction.create('" +pipe+ "') "
        )
        result = tx.run(query)
        
    @staticmethod
    def _add_node_props(tx, pipe, prop, config):
        query = (
            "CALL gds.beta.pipeline.linkPrediction.addNodeProperty('" +pipe+  "', '" +prop+ "'," +config+ ") YIELD nodePropertySteps "
        )
        result = tx.run(query)
        
    @staticmethod
    def _add_link_feat(tx, pipe, feature, config):
        query = (
            "CALL gds.beta.pipeline.linkPrediction.addFeature('" +pipe+ "', '"+feature+"' , "+config+") YIELD featureSteps; "
        )
        
        
        result = tx.run(query)
    
    @staticmethod
    def _split_data(tx, pipe):
        query = (
            "CALL gds.beta.pipeline.linkPrediction.configureSplit('"+ pipe+ """',
            { testFraction: 0.20, trainFraction: 0.80, negativeSamplingRatio: 1.0}) YIELD splitConfig;   
            """
        )
        result = tx.run(query)
    
    @staticmethod    
    def _add_model_cands(tx, pipe, param):
        query = (
            "CALL gds.alpha.pipeline.linkPrediction.addRandomForest('"+ pipe+"', "+param+") YIELD parameterSpace; " 
        )
        result = tx.run(query)
    
    @staticmethod
    def _train_model(tx, proj, pipe):
        query = (
            "CALL gds.beta.pipeline.linkPrediction.train('" +proj+ "', { pipeline: '"+pipe+ """',
            modelName: 'rf-pipeline-model3',  concurrency: 1, randomSeed: 99 }) YIELD modelInfo  
            RETURN modelInfo.bestParameters AS winningModel, modelInfo.metrics.AUCPR.train.avg AS avgTrainScore,
            modelInfo.metrics.AUCPR.outerTrain AS outerTrainScore, modelInfo.metrics.AUCPR.test AS testScore 
            """
        )
        result = tx.run(query)
        try:
            return [{"winningModel": row["winningModel"], 
                     "avgTrainScore": row["avgTrainScore"],
                    "outerTrainScore": row["outerTrainScore"],
                    "testScore": row["testScore"]} for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise
            
    def LinkPrediction(self, proj, pipe, props, config_n, features, config_f, model_params):
        with self.driver.session() as session:
            session.write_transaction(self._creat_projection, proj)
            session.write_transaction(self._make_pipe, pipe)
            
            for i in range(len(props)):
                session.write_transaction(self._add_node_props, pipe, props[i], config_n[i])
                
            for i in range(len(features)):
                session.write_transaction(self._add_link_feat,pipe, features[i], config_f[i])
            
            session.write_transaction(self._split_data, pipe)
            
            for i in range(len(model_params)):
                session.write_transaction(self._add_model_cands, pipe, model_params[i])   
                
            result = session.write_transaction(self._train_model, proj ,pipe)
        print(result)
        
    @staticmethod
    def _predict_rels(tx, proj, model_name):
        query = (
            "CALL gds.beta.pipeline.linkPrediction.predict.mutate('"+proj+"', { modelName: '"+model_name+"', mutateRelationshipType: 'Predicted_Relation', threshold: 0.95,  topN: 10, concurrency: 1, randomSeed:20}) YIELD relationshipsWritten, samplingStats"
            " Return relationshipsWritten, samplingStats"
        )
        result = tx.run(query)
        return [{"relationshipsWritten": row["relationshipsWritten"], 
                     "samplingStats": row["samplingStats"] } for row in result]
        
    def predict(self, proj, model_name):
        with self.driver.session() as session:
            result = session.write_transaction(self._predict_rels, proj, model_name)
                
        print(result)
            
    @staticmethod
    def _drop_model(tx, model):
        query = (
            "CALL gds.beta.model.drop('"+model+"')"
        )
        result = tx.run(query)
        
    def drop_model(self, model):
        with self.driver.session() as session:
            result = session.write_transaction(self._drop_model, model)
            
    @staticmethod
    def _drop_proj(tx, proj):
        query = (
            "CALL gds.graph.drop('"+proj+"')"
        )
        result = tx.run(query)
        
    def drop_proj(self, proj):
        with self.driver.session() as session:
            result = session.write_transaction(self._drop_proj, proj)
            
    @staticmethod
    def _drop_pipe(tx, pipe):
        query = (
            "CALL gds.beta.pipeline.drop('"+pipe+"')"
        )
        result = tx.run(query)
        
    def drop_pipe(self, pipe):
        with self.driver.session() as session:
            result = session.write_transaction(self._drop_pipe, pipe)
        

# Creating connection and importing relations:

In [23]:
# Adding new nodes usnig Reference code My edition:
for i, row in relations.iterrows():
    app.create_actors_relationship_with_usecase(row["source"], row["message"],
                                                row["target"], row["id"])
# importing functions:
import json

with open("funcs.json") as f:
    funcs = json.loads(f.read())
    
for func in funcs:
    if funcs[func] in edits:
        funcs[func] = edits[funcs[func]]
# create json object from dictionary
funcs['Monitor and analyze the experimentation'] = 'AN Sandbox'
json = json.dumps(funcs)

# open file for writing, "w" 
f = open("newfuncs.json","w")

# write json object to file
f.write(json)

# close file
f.close()
comps_w_funcs = {}

comps = set(funcs.values())
for comp in comps:
    comps_w_funcs[comp]=[]
    for f in funcs:
        if funcs[f] == comp:
            comps_w_funcs[comp].append(f)
for c in comps_w_funcs:
    str_ = ", ".join(comps_w_funcs[c])
    app.writePropToNode(c, "capability", str_)


Created node: Knowledge base
Created node: AN component-N
Created relation: Knowledge base - exchange of knowledge - AN component-N in UC001
Created node: AN Component-1
Node already exists: AN component-N in ['UC001']
ignoring ...
Created relation: AN Component-1 - exchange of knowledge - AN component-N in UC001
Node already exists: Knowledge base in ['UC001']
ignoring ...
Node already exists: AN Component-1 in ['UC001']
ignoring ...
Created relation: Knowledge base - Report - AN Component-1 in UC001
Node already exists: Knowledge base in ['UC001']
ignoring ...
Node already exists: AN Component-1 in ['UC001']
ignoring ...
Created relation: Knowledge base - exchange of knowledge - AN Component-1 in UC001
Created node: Management domain
Node already exists: Knowledge base in ['UC001']
ignoring ...
Created relation: Management domain - import of knowledge - Knowledge base in UC001
Node already exists: Management domain in ['UC001']
ignoring ...
Node already exists: Knowledge base in ['UC

Created relation: End-user communities - Network connectivity option-2 - Industry vertical applications @ edge in UC011
Node already exists: End-user communities in ['UC011']
ignoring ...
Node already exists: Industry vertical applications @ edge in ['UC011']
ignoring ...
Created relation: End-user communities - Network connectivity option-3 - Industry vertical applications @ edge in UC011
Node already exists: End-user communities in ['UC011']
ignoring ...
Node already exists: AN in ['UC010', 'UC011']
ignoring ...
Created relation: End-user communities - Manage end-user onboarding, management, billing, etc - AN in UC011
Created node: NAO @ edge
Node already exists: AN Component-1 in ['UC001', 'UC002', 'UC003', 'UC004']
adding label UC020 in addition to ['UC001', 'UC002', 'UC003', 'UC004']
Created relation: NAO @ edge - Closed loop report based on abstracted network and app state - AN Component-1 in UC020
Node already exists: AN Component-1 in ['UC001', 'UC002', 'UC003', 'UC004', 'UC020

Created relation: AN orchestrator - Learn new classes of players, strategies and equilibrium - CL automation framework in UC032
Node already exists: OpenCN (repo) in ['UC030', 'UC031', 'UC032']
ignoring ...
Node already exists: AN orchestrator in ['UC030', 'UC031', 'UC032']
ignoring ...
Created relation: OpenCN (repo) - Model initial game and players - AN orchestrator in UC032
Node already exists: OpenCN (repo) in ['UC030', 'UC031', 'UC032']
ignoring ...
Node already exists: AN orchestrator in ['UC030', 'UC031', 'UC032']
ignoring ...
Created relation: OpenCN (repo) - Controller upload - AN orchestrator in UC032
Node already exists: OpenCN (repo) in ['UC030', 'UC031', 'UC032']
ignoring ...
Node already exists: AN orchestrator in ['UC030', 'UC031', 'UC032']
ignoring ...
Created relation: OpenCN (repo) - Trust grades are established - AN orchestrator in UC032
Node already exists: OpenCN (repo) in ['UC030', 'UC031', 'UC032']
ignoring ...
Node already exists: AN orchestrator in ['UC030', 'U

Created relation: AN Sandbox - evaluation results of control loop apps - Controller in UC039
Node already exists: Controller repository in ['UC039']
ignoring ...
Node already exists: Controller in ['UC004', 'UC039']
ignoring ...
Created relation: Controller repository - availability response(details of control loop(could be more than one) like size, features, requirements) - Controller in UC039
Node already exists: AN_1 in ['UC038', 'UC039']
ignoring ...
Node already exists: Controller in ['UC004', 'UC039']
ignoring ...
Created relation: AN_1 - negative response - Controller in UC039
Node already exists: Controller in ['UC004', 'UC039']
ignoring ...
Node already exists: Controller repository in ['UC039']
ignoring ...
Created relation: Controller - Command to download the selected control loop apps in a sandbox environment - Controller repository in UC039
Created node: AN_2
Node already exists: AN_1 in ['UC038', 'UC039']
adding label UC040 in addition to ['UC038', 'UC039']
Created relat

# Training and predicting:

In [154]:
## Making link prediction:
props = ["fastRP", 'degree', 'labelPropagation']
conf_n = ["{ mutateProperty: 'embedding', embeddingDimension: 64, randomSeed: 20}", 
          "{ mutateProperty: 'degree' }", 
         "{mutateProperty: 'community'}"]

features = ['l2', 'cosine', 'l2', "l2"]
config_f = ["{ nodeProperties: ['embedding'] }" ,
            "{ nodeProperties: ['embedding']}"  ,
            "{ nodeProperties: ['degree'] }"    , 
            "{ nodeProperties: ['community'] }"]

model_params = ['{numberOfDecisionTrees: 2, maxDepth: 15, numberOfSamplesRatio: 0.9}',
                "{numberOfDecisionTrees: 5, maxDepth: 100 , numberOfSamplesRatio: 0.5}"]

app.drop_model('rf-pipeline-model3')
app.drop_proj('proj95')
app.drop_pipe('pipe95')
app.LinkPrediction("proj95", "pipe95", props, 
                   conf_n, features, config_f,
                   model_params)

#Export prjected Grpah
# CALL gds.graph.export("proj95", {dbName: "preditedGrpah"})
# :use system;
# CREATE DATABASE preditedGrpah;

# :use preditedGrpah
# MATCH (n) RETURN n;

[{'winningModel': {'maxDepth': 15, 'minSplitSize': 2, 'numberOfDecisionTrees': 2, 'methodName': 'RandomForest', 'numberOfSamplesRatio': 0.9}, 'avgTrainScore': 0.9171934400236287, 'outerTrainScore': 0.932149751792609, 'testScore': 0.8890819535884231}]


In [157]:
uri = "bolt://localhost:7687"
user = "neo4j"
password = "123"
app = App(uri, user, password)

In [21]:
# app.cleanup_db()

In [158]:
app.predict('proj95', 'rf-pipeline-model3')

[{'relationshipsWritten': 20, 'samplingStats': {'linksConsidered': 936, 'strategy': 'exhaustive'}}]
